In [73]:
import tensorflow as tf

# load the model
model = tf.keras.models.load_model('unet_model')

In [74]:
# load the test data
import os

data_dir = 'data'
mask_dir = "external_contours_masks"
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
test_masks_dir = os.path.join(mask_dir, 'test')
train_masks_dir = os.path.join(mask_dir, 'train')

# Create a folder with results and subfolders for cropped images, heatmaps and dijkstra
results_dir = 'external_contours_masks/results'
cropped_dir = os.path.join(results_dir, 'cropped')
heatmaps_dir = os.path.join(results_dir, 'heatmaps')
dijkstra_dir = os.path.join(results_dir, 'dijkstra')

if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    os.mkdir(cropped_dir)
    os.mkdir(heatmaps_dir)
    os.mkdir(dijkstra_dir)

In [80]:
# run over the test data and predict the masks
# Find the contour of the mask
# Crop the image according to the contour

import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_contour(mask: np.ndarray):
     # Apply thresholding
    ret, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    # sDefine structuring element (MORPH_RECT)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    # Erosion using MORPH_RECT structuring element
    erosion= cv2.erode(thresh, kernel, iterations=1)
    # Dilate using MORPH_RECT structuring element
    dilation = cv2.dilate(erosion, kernel, iterations=1)
    # Find contour using the eroded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Find the contour with the largest area
    contour = max(contours, key=cv2.contourArea)
    return contour

def crop_image(img, contour):
    # Resize the contour to the original image size from 128x128 to 4096x4096
    # Add erosion to the contour to make sure the mask will cover the whole tumor
    contour = 32 * contour
    # Create a mask with the contour
    mask = np.zeros(img.shape, np.uint8)
    cv2.drawContours(mask, [contour], -1, (255, 255, 255), -1, cv2.LINE_AA)
    # Apply the mask to the image
    img = cv2.bitwise_and(img, mask)
    return img


def predict_mask(img):
    # Predict the mask
    img = cv2.resize(img, (128, 128))
    img = np.expand_dims(img, axis=0)
    img = np.expand_dims(img, axis=-1)
    img = img / 255
    pred = model.predict(img)
    pred = tf.argmax(pred, axis=-1)
    pred = pred[..., tf.newaxis]
    pred = tf.squeeze(pred)
    pred = pred.numpy()
    pred = pred.astype(np.uint8) * 255
    return pred

def get_mask(img):
    # Find the mask
    mask = predict_mask(img)
    # Find the contour of the mask
    contour = get_contour(mask)
    # Crop the image according to the contour
    img = crop_image(img, contour)
    return img

def get_masked_image(img):
    # Find the mask
    img = get_mask(img)
    img[img == 0] = 0
    return img

directories = [test_dir, train_dir]
for directory in directories:
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        img = get_masked_image(img)
        # Write the image to the results/cropped folder
        img_path = os.path.join(cropped_dir, img_name)
        cv2.imwrite(img_path, img)

1/1 [==============================] - 0s 63ms/step


In [95]:
# Create heatmaps of the magnitude of the gradient
def get_heatmap(img, mask):# Sliding window over edges image in two dimensions to create a heatmap of the edges
    # apply threshold to binarize the image
    _, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(thresh, (13,13), 0)
     # Apply Canny edge detection
    edges = cv2.Canny(blur, 50, 150)
    window_size = 201 # Odd number only
    window_step = 10
    heat_map = np.zeros_like(edges)
    # pad the images with zeros
    edges_c = np.pad(edges, int((window_size-1)/2), mode='constant', constant_values=0)
    mask_metal_c = np.pad(mask, int((window_size-1)/2), mode='constant', constant_values=0)
    for y in range(0, edges_c.shape[0], window_step):
        for x in range(0, edges_c.shape[1], window_step):
            window = edges_c[y:y+window_size, x:x+window_size]
            mask_metal_window = mask_metal_c[y:y+window_size, x:x+window_size]/255
            if mask_metal_window.sum() == 0 or mask_metal_window[(int((window_size - 1) / 2)), (int((window_size - 1) / 2))] == 0:
                heat_map[y:y+window_step, x:x+window_step] = 0
            else:
                heat_map[y:y+window_step, x:x+window_step] = np.sum(window)/mask_metal_window.sum()

    heat_map = cv2.equalizeHist(heat_map)
    return heat_map

for img_name in os.listdir(cropped_dir):
    img_path = os.path.join(cropped_dir, img_name)
    img = cv2.imread(img_path)
    # Find the mask
    mask = get_masked_image(img)
    # Find the heatmap
    heatmap = get_heatmap(img, mask)
    # Write the image to the results/heatmaps folder
    img_path = os.path.join(heatmaps_dir, img_name)
    cv2.imwrite(img_path, heatmap)

1/1 [==============================] - 0s 72ms/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()